In [44]:
import fingertips_py as ftp
from json import loads
import pandas as pd
from urllib.request import urlopen

In [45]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

- A unique data set is ['Indicator Name', 'Indicator ID', 'Sex', 'Age', 'Time period', 'AreaTypeId]

- Need to create template table of indictor + area codes so that shape shows in QS

- The indicator name needs to be name + id

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

- time period
- sex
- age

# time period, sex and age available for each indicator

# Area types available for each indicator

In [46]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

# Indicator metadata
To get the name of each indicator

In [47]:
indicator_metadata = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/csv/all')
# indicator_metadata.columns = indicator_metadata.columns.str.lower().str.replace(' ', '_')


# Indicator sex, age and timeperiod available

This doesn't actually bring through all of those available in the data. It brings through the settings for the website and what is shown on the website

In [48]:
def df_nested_json(url): 
    with urlopen(url) as response:
        data_str = response.read().decode("utf-8")
        data_json = loads(data_str[1:-1])
    return pd.json_normalize(data_json)

In [49]:
data = df_nested_json('https://fingertips.phe.org.uk/api/grouproot_summaries/by_indicator_id?indicator_ids=90813')

In [50]:
data

,IndicatorName,IID,GroupId,StateSex,StateAge,StateYearRange,TimePeriod,YearRange,Sex.Id,Sex.Name,Age.Id,Age.Name,Age.Sequence,Category.Id,Category.CategoryTypeId,Category.Name,Category.ShortName,DateChanges.HasDataChangedRecently,DateChanges.DateOfLastChange,DataQualityNote.Id,DataQualityNote.Text,DataQualityNote.Comment,Unit.Id,Unit.Value,Unit.Label,ValueType.Id,ValueType.Name
0,Hospital admissions as a result of self-harm (...,90813,1938132923,False,False,False,2021/22,1,4,Persons,305,10-24 yrs,17.0,-1,-1,Undefined,Undefined,False,07 Mar 2023,0,None,None,3,100000.0,"per 100,000",1,Directly standardised rate


# Area Ids metadata
To get the name of the area

In [51]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

## hierarchy

In [52]:
area_hierarchy = pd.read_csv('https://docs.google.com/spreadsheets/u/0/d/15RhWWsHPPMLWoxR5sJcpK-vraRkidRY8jsAb_Y_5GwI/gviz/tq?tqx=out:csv&tq&gid=963757659&headers=1')
area_hierarchy = area_hierarchy[['AreaTypeId', 'map_position', 'hierarchy']].dropna()

# Area types available for each indicator with extra metadata

## prepare full table

In [53]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True).rename({'_merge': 'area_merge'}, axis=1)

available_indicator_at_area = pd.merge(left=available_indicator_at_area, right=indicator_metadata.loc[:,['Indicator ID', 'Indicator']],
                                       left_on='IndicatorId', right_on='Indicator ID', how='inner',
                                       indicator=True, )

## use hierarchy to choose indicators and areas

In [54]:
full_area_indicator_hierarchy = pd.merge(left=area_hierarchy, right=raw_available_indicator_at_area,
         left_on='AreaTypeId', right_on='AreaTypeId',
         how='inner')

In [55]:
left = full_area_indicator_hierarchy[full_area_indicator_hierarchy['map_position']=='left']
left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')
keep_left = left[left['rank']==1]


In [56]:
keep_left['rank'].unique()

array([1.])

In [57]:
def find_first_hierarchy(data, map_position):
    data = data[data['map_position']==map_position].loc[:, ['IndicatorId','hierarchy','AreaTypeId', 'map_position']]
    data['rank'] = data.groupby('IndicatorId')['hierarchy'].rank(method='dense')
    return data[data['rank']==1].drop('rank', axis=1)

In [58]:
get_indicator_area = pd.concat([find_first_hierarchy(full_area_indicator_hierarchy, 'left'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'middle'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'right')]
)

# Retrieve indicator data

## filter by those that only have all 3 as a test case

In [59]:
number_of_maps_returned = get_indicator_area.groupby('IndicatorId')['map_position'].count()
ind_ids_keep_3_maps = number_of_maps_returned[number_of_maps_returned==3].index.values

In [60]:
get_ind_area_combos = get_indicator_area[get_indicator_area['IndicatorId'].isin(ind_ids_keep_3_maps)]

## get data

In [61]:
def return_ind_list_for_area_type(data, AreaTypeId):
    return data[data['AreaTypeId']==AreaTypeId]['IndicatorId'].values

In [62]:
return_ind_list_for_area_type(get_ind_area_combos, 221)

array([], dtype=int64)

In [63]:
def get_data(data, area_type_id):
    indicator_list = return_ind_list_for_area_type(data, area_type_id)
    
    replace_comma = '%2C'
    ids_as_str = replace_comma.join([str(i) for i in indicator_list])
    print(ids_as_str)
    data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
                                                    area_type_id=area_type_id,
                                                    # parent_area_type_id=15,
                                                    # profile_id=None,
                                                    include_sortable_time_periods=True,
                                                    is_test=False)
    
    return data
    
    # data.to_csv(str(area_type_id)+'.csv')

In [64]:
# for area_id in get_ind_area_combos['AreaTypeId'].unique():
#     get_data(get_ind_area_combos, area_id)

# Analysis of nulls

In [66]:
area_ref[area_ref['Class'].str.contains('ccg-composite')]

,Id,Name,Short,Class,Sequence,CanBeDisplayedOnMap
51,152,CCGs (2017/18),CCGs (2017/18),ccg-composite,2017,True
52,154,CCGs (2018/19),CCGs (2018/19),ccg-composite,2018,True
62,165,CCGs (2019/20),CCGs (2019/20),ccg-composite,2019,True
63,166,CCGs (2020/21),CCGs (2020/21),ccg-composite,2020,True
64,167,CCGs (2021/22),CCGs (from Apr 2021),ccg-composite,2021,True


In [67]:
analyse_nulls_data = get_data(get_ind_area_combos, 165)
print(analyse_nulls_data['Value'].isna().sum())
# remove null Values
analyse_nulls_data = analyse_nulls_data[~analyse_nulls_data['Value'].isna()]
print(analyse_nulls_data['Value'].isna().sum())

# remove England
print(analyse_nulls_data['Area Name'].nunique())
analyse_nulls_data = analyse_nulls_data[analyse_nulls_data['Area Name']!='England']
print(analyse_nulls_data['Area Name'].nunique())


c:\Users\medssam\Documents\GitHub\Health_Indicators\.venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fingertips.phe.org.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ParserError: Error tokenizing data. C error: Expected 1 fields in line 14, saw 2


In [ ]:
analyse_nulls_data['Indicator Name'].nunique()

In [ ]:
analyse_nulls_data.columns

In [ ]:
# analyse_nulls_data.groupby(['Indicator Name', 'Indicator ID', 'Sex', 'Age', 'Time period'])['Area Code'].nunique()

# Analysis of indicator ids and names

Some indicator ids have the same name - do they have the same metadata?

No, they have different metadata so a unique indicator would be (indicator id + name)

In [ ]:
indicator_ids_per_name = pd.DataFrame(indicator_metadata.groupby('Indicator')['Indicator ID'].nunique()).reset_index()


In [ ]:
indicators_with_same_name = indicator_ids_per_name[indicator_ids_per_name['Indicator ID']>1]

In [ ]:
indicators_with_same_name.head()

In [ ]:
indicator_name = indicators_with_same_name['Indicator'].iloc[0]
indicator_metadata[indicator_metadata['Indicator']==indicator_name]